# Dog Breed Classification Project Proposal

### Background
The project focus on using deep learning for image classification problem. Image classification is a type of supervised learning, where the input is an image (can be represented as a 2D/3D feature array) and the output is the probablility of the image beloning to certain class. Convolutional neural network (CNN) has been proven as an effective method for this problem. One popular image classification challenge in the ImageNet contest: ImageNet Large Scale Visual Recognition Challenge (ILSVRC)$^{[1]}$. The challenge is to correctly classify 1000 classes. Winners of the ILSVRC include several break through in CNN architecture: AlexNet(2011)$^{[2]}$, VGG16(2014)$^{[3]}$, ResNet50(2015)$^{[4]}$. These architectures are often used as pretrained model in transfer learning.

### Project statement

In this project, we will build a neural network to identify dog breed from a supplied dog image. In specific, when given an arbitary image, the algorithm should first identify if it is a dog or a human. If dog is detected, the code will return the estimate of breed. If human is detected, the code will return the resembling dog breed.

I choose this project because I find deep learning is very facinating. And I want to gain experience in building neural network to solve a practical problem.

### Data Set

Two data set are provide by Udacity.
(1) Dog data set (https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip):
    * 8351 dog images of 133 dog breed categories
    * split into train, validation and test set, with 6680, 835, 836 images respectively.
    * In training set, each class has 30-70 images.
    
![](eda/number_of_images_per_class.png)
    
(2) Human data set (http://vis-www.cs.umass.edu/lfw/lfw.tgz):
    * 13233 human face images
    
    
Images in both dog and human dataset are colored, so the input data will be nW * nH * nC where nC=3. If we are going to detecting faces, gray scale image might be enough, but for identifying dog breed, the color may be necessary. A sample image from dog and human dataset is shown below:
    
|Sample Dog Image     |Sample Human Image |
:-------------------------:|:-------------------------:
|<img src="eda/Australian_cattle_dog_00756.jpg" height="400" />|<img src="eda/Charles_Chandler_IV_0001.jpg" height="400" /> 

### Proposed Solution

For detecting human or dog, we can use either image classification or object detection for this problem. As this is a pretty standard task, there are some existing implementations we can use directly. For example, we can use OpenCV's face detector for human detection.

For dog breed classification, we can either train a nerual network from scratch or use transfer learning. The architecture neural network of image classification typically consists of two part: feature extraction part and classifier part. The feature extraction part usually consists of a series of convolutional layers. The classifier usually consists of a few fully-connected/linear layers and the final layer would out put N_class numbers that can be used to compute loss function. For transfer learning, we can use a pretrained model, e.g. VGG16, ResNet50, change the last layer to match the number of classis in our problem. We will fix the weight of feature extraction part, and fine tune the classifier part.

### Benchmark Model

For dog breed classification, I tried to use AWS Sagemaker's build in image classification. I choose to use pretrained 18 layer network and fine tune the last layer. However, I can only archieve 1% accuracy on both training and validation, and early stopping was triggered before 30 epochs. The algorithm is kind of black-box to me and I am not sure how to improve it.

Below are my code to start a image classification training job on Sagemaker:

In [ ]:
ic = sagemaker.estimator.Estimator(
                training_image,
                role, 
                train_instance_count=1, 
                train_instance_type='ml.p2.xlarge',
                train_volume_size = 50,
                train_max_run = 360000,
                input_mode= 'File',
                output_path=s3_output_location,
                sagemaker_session=sess,
                base_job_name='project-dog',
                    )
ic.set_hyperparameters(num_layers=18,
                             use_pretrained_model=1,
                             image_shape = "3,224,224",
                             num_classes=133,
                             num_training_samples=6678,
                             mini_batch_size=32,
                             epochs=30,
                             learning_rate=0.001,
                             precision_dtype='float32',
                             early_stopping=True,
                             optimizer='adam',
                             checkpoint_frequency=5,
                    )

### Evaluation Metrics

For dog/human detection, we can evaluate the accuracy. We also would like to focus on false positive and false negative rate. For example, given a set of human images, the human detector can failed to detect how many images. Given a set of dog images, how many images the human detector may mistakenly detect dog as human.

For dog breed classification, as this is a multiclass classification, we can use cross-entropy as optimization and evaluation metric. We can also use accuracy as evaluation metric.

Cross-entropy Loss: $L = -\frac{1}{N}\sum_i^{N_{sample}} \sum_c^{N_{class}} t_{ic} log (\hat{y}_{ic})$

Accuracy: $\frac{\text{number of correctly classified samples}}{\text{number of samples}}$

### Project Design

The project workflow is as follows:

(1) Dog detection and human detection

This part we will use existing algorithms. There isn't much things we can tune for a given algorithm. However, we should evaluate the performance on a given set of images and make sure the performance is satisfactory. We should also compare multiple algorithms.

(2) Dog Breed Classification

We will train our own neural network for this part, whether from scratch or using transfer learning. The training consists of the following steps:

* Read images
* Preprocess images. This step is includes resizing all training images to the same size, rescale the pixels in the range that is excepted for pretrained model.
* Data Augmentation. We can use random flip or rotation to augment the dataset. Pytorch does the augmentation on the fly conveniently: random augmentation is performed each time when a batch of image is loaded.
* Training: weight is adjusted according to gradient descent. The validation dataset is used to monitor the evaluation metric during training process. Best model is saved according to the performance on validation dataset.
* Testing: after loading the best model, we can evaluate metrics on the test dataset.

(3) Combine Step 1 and 2

We will combine the model saved from 1 and 2 to design a deliverable function that achieve the project goal: predict dog breed given a dog image or identify a resembling dog breed given a human image. We will test behavior the final function using images provided in the dataset as well as some images find by myself.

### References

[1] Russakovsky, Olga, et al. "Imagenet large scale visual recognition challenge." International journal of computer vision 115.3 (2015): 211-252.

[2] Krizhevsky, Alex; Sutskever, Ilya; Hinton, Geoffrey E. (2017-05-24). "ImageNet classification with deep convolutional neural networks" (PDF). Communications of the ACM. 60 (6): 84–90. doi:10.1145/3065386. ISSN 0001-0782.

[3] Simonyan, Karen, and Andrew Zisserman. "Very deep convolutional networks for large-scale image recognition." arXiv preprint arXiv:1409.1556 (2014).

[4] He, Kaiming, et al. "Deep residual learning for image recognition." Proceedings of the IEEE conference on computer vision and pattern recognition. 2016.